# Practica 2 - Familia de Columnas

## Plataforma de libros

**Integrantes**
- Avitúa Varela Fernando
- Santa Rita Vizuet Fernando
- Andrés Urbano Guillermo 
---
Desarrollar una plataforma de libros.

- Debe incluir producto, cliente y calificación. Cada cliente debe tener información asociada (nombre, país, membresía, etc.)

- Además, cada cliente debe poder elegir una categoría para cada uno de sus libros (fantasía, misterio, etc.). Todos deben participar como clientes.

Como administradores, deben poder buscar cosas como:
- La categoría preferida de un cliente dado.

- Obtener los clientes que más disfrutaron un libro dado.

- Los mejores libros de una categoría dada.


### Diseño del modelo para la Plataforma de libros

Primero, importaremos el driver de Cassandra para poder conectarnos a la base de datos.

In [1]:
from cassandra.cluster import Cluster

# Cambiar host si es necesario
host = '172.17.0.2'
keyspace = 'cheetah_books'

# Conexion al servidor de Cassandra y al Keyspace 
cluster=Cluster([host],port=9042)
session=cluster.connect(keyspace)

Una vez conectado, cremos nuestra primera tabla de clientes, dado que tenemos información del pais y membresia. Con esto, podremos consultar información de un usuario dado nombre.


In [2]:
session.execute('DROP TABLE IF EXISTS clientes;')

# Creación de tabla de clientes
session.execute("""CREATE TABLE clientes (
	nombre TEXT,
	pais TEXT,
	membresia TEXT,
	PRIMARY KEY ((nombre), pais)
);""");

Se crea una función para leer un diccionario (cada entrada del csv) y producir una consulta de insert

In [3]:
def datos_a_insert_query(nombre_tabla, diccionario, columnas=False):
    """
    Regresa un string con el query que agrega el registro indicado en diccionario, con las columnas especificadas

    Parametros
    ----------
    nombre_tabla : str
        El nombre de la tabla donde se va a insertar

    diccionario: dict
      Un diccionario que tiene la información del registro a ingresar, con el formato {nombre_columna_1:valor_a_insertar_1,...}

    columnas: list(str)       
      lista del nombre de las columnas a insertar. 

    Regresa
    -------
    salida : str
        Query en cql usado para insertar los valores especificados
    """

    if not columnas:
        columnas = list(diccionario.keys())
    values = []
    keys = []

    for k, v in diccionario.items():
        if k in columnas:
            keys.append(k)
            if type(v) == int:
                values.append(str(v))
            elif type(v) == str:
                # para evitar problemas con Ender's Game o alguno que tenga más espacios o diferente capitalización
                v1 = v.replace("'", '').lower().strip()
                values.append(f"'{v1}'")
    return f"INSERT INTO {nombre_tabla} ({','.join(keys)}) VALUES ({','.join(values)});"

Esta función nos ayudara a lo largo del notebook para ir ingresando los valores a nuestras tablas a partir del csv generado por el grupo.

Leemos los datos de los clientes:

In [4]:
import pandas as pd

df = pd.read_csv('Clientes_plataforma - Hoja 1.csv', names=['nombre', 'pais', 'membresia'],
               header=0)
df.head()

,nombre,pais,membresia
0,Yeudiel,Mexico,Free
1,Eduardo,Mexico,Basica
2,FernandoA,Mexico,Basica
3,Edgar,Mexico,Free
4,Daniel,Mexico,Free


Una vez leido nuestro csv, ingresaremos los valores a nustra tabla a traves de nuestro función creada anteriormente:

In [5]:
for row in df.iterrows():
    query = datos_a_insert_query('clientes',row[1].to_dict(),
                                 columnas=['nombre', 'pais', 'membresia'])            
    session.execute(query)

Con esto podemos obtener la información de cualquier cliente dado en nuestro aplicación:

In [6]:
nombre_cliente = "hugo"
query = f"""
    SELECT * FROM clientes
    WHERE nombre = '{nombre_cliente}';
    """
response = session.execute(query)
response.one()

Row(nombre='hugo', pais='mexico', membresia='premium')

Ahora procederemos a leer la información de los libros:

In [7]:
df=pd.read_csv('DatosBBDDE - Hoja 1.csv', names=['categoria', 'usuario', 'libro', 'calificacion'],
               header=0)
df.head()

,categoria,usuario,libro,calificacion
0,Fantasia,Yeudiel,Harry Potter,5
1,Suspenso,Eduardo,One Shot,5
2,Novela,FernandoA,On the road,5
3,Novela,Edgar,The Pillars of the Earth,4
4,Realismo mágico,Daniel,100 años de soledad,5


## Consulta 1 

La categoría preferida de un cliente dado.

Primero, creamos la tabla `categorias_por_usuario` que necesitamos para esta consulta.

In [8]:
session.execute('DROP TABLE IF EXISTS categorias_por_usuario;')

# tabla categories_by_user
session.execute("""CREATE TABLE categorias_por_usuario (
                usuario text,
                libro text,
                categoria text,
                calificacion float,
                PRIMARY KEY((usuario, categoria), calificacion, libro))""");

Del conjunto de datos recolectado en clase (guardado como `df`, DataFrame en Pandas), los agregamos a la tabla anterior.

In [9]:
for i in df.iterrows():
    session.execute(f"""INSERT INTO categorias_por_usuario (
                    usuario,
                    libro,
                    categoria,
                    calificacion)
                    VALUES ('{str(i[1][1])}',
                    '{str(i[1][2])}',
                    '{str(i[1][0])}',
                    {str(i[1][3])})""")

Antes de definir la función que realiza la consulta en la base de datos, debemos establecer los criterios para que una categoría sea la favorita de un usuario.

**Criterios de asignación de la categoría favorita de un usuario**

1. Decimos que una categoría se prefiere sobre otra si el promedio de calificación de los libros de dicha categoría es mayor que las demás.

2. En caso de que haya promedios iguales entre dos categorías, entonces establecemos como criterio de desempate elegir la categoría con más libros calificados.

3. Si existe un empate después de los dos filtros anteriores, entonces es no existe criterio para escoger una categoría u otra. Por tanto, permanece la categoría líder hasta el momento.

Definimos entonces la función encargada de realizar la consulta sobre esta tabla. 

In [10]:
def obtener_categoria_preferida(usuario) -> str:
    """
    Regresa la categoría que tiene mejor calificación en promedio para un cliente dado

    Parametros
    ----------
    cliente : str
        El cliente al que le buscaremos su categoría

    Regresa
    -------
    salida : str
        El nombre de la categoría favorita (en promedio)
    """
    
    # consulta en CQL
    consulta = session.execute(f"""SELECT categoria, AVG(calificacion), COUNT(libro)
                                FROM categorias_por_usuario
                                WHERE usuario = '{usuario}'
                                GROUP BY categoria
                                ALLOW FILTERING""")
    
    # variable que nos indica la categoria con mayor promedio hasta el momento
    avg_max = 0
    
    # categoria con mejor promedio
    cat_max = 'No se obtuvieron resultados'
    
    # cantidad de libros de la categoría
    count_max = 0
    
    # filtramos a la categoría preferida bajo los siguientes criterios
    for i in consulta:

        # criterio de asignación
        if avg_max < i[1]:
            cat_max, avg_max, count_max = i[0], i[1], i[2]
            
        # criterio de desempate
        elif avg_max == i[1]:
            if count_max < i[2]:
                cat_max, avg_max, count_max = i[0], i[1], i[2]
            
    return f'La categoría preferida de {usuario} es {cat_max}'

In [11]:
obtener_categoria_preferida('Lucas')

'La categoría preferida de Lucas es Cuento'

**Prueba**
```
obtener_categoria_preferida('Lucas')
```
**Resultado**
```
'La categoría preferida de Lucas es Cuento'
```

## Consulta 2 

Los clientes que más disfrutaron un libro dado.

Se crea la tabla usada para los queries. Es necesario poner la calificación como llave de cluster para poder ordenar con ella

In [12]:
session.execute('DROP TABLE IF EXISTS clientes_por_libro;')


session.execute('''CREATE TABLE clientes_por_libro (
libro text,
usuario text,
calificacion int,
PRIMARY KEY ((Libro),calificacion,usuario)
);''');

Se hacen las inserciones

In [13]:
for row in df.iterrows():
    q=datos_a_insert_query('clientes_por_libro',row[1].to_dict(),columnas=['usuario','calificacion','libro'])
    session.execute(q)

Se crea la consulta y se muestran los resultados 

In [14]:
def obtener_fanaticos(libro):
    """
    Obtiene los clientes que más disfrutaron un libro dado

    Parameters
    ----------
    book: Book
        El libro
    Returns
    -------
    clientes : list
        Lista de clientes 
    """
    # Clientes que más disfrutaron un libro
    q2=f'''
    SELECT usuario,calificacion
    FROM clientes_por_libro
    WHERE libro='{libro}'
    ORDER BY calificacion DESC LIMIT 5
    '''
    resQuery=session.execute(q2)
    s=f"Clientes que más disfrutaron el libro:\n{libro}"
    print(s)
    print('-'*len(s))
    print(f"{'Usuario':{len(s)-17}}| {'Calificacion':{14}}|")
    print('-'*len(s))
    for ai in resQuery:
        print(f'{ai.usuario:{len(s)-17}}| {ai.calificacion:14}|')

    return resQuery

Probamos ahora nuestra función:

In [15]:
obtener_fanaticos('siddharta');

Clientes que más disfrutaron el libro:
siddharta
------------------------------------------------
Usuario                        | Calificacion  |
------------------------------------------------
fernandos                      |              5|
fernandoa                      |              3|


**Resultado esperado**
```
Clientes que más disfrutaron el libro:
siddharta
------------------------------------------------
Usuario                        | Calificacion  |
------------------------------------------------
fernando                       |              5|
fernandoa                      |              3|
```

## Consulta 3  Los mejores libros de una categoría dada

Creación de la tabla `libros_por_categoria`

In [16]:
session.execute('DROP TABLE IF EXISTS libros_por_categoria;')

session.execute('''CREATE TABLE libros_por_categoria (
	categoria TEXT,
	usuario TEXT,
	libro TEXT,
	calificacion INT,
	PRIMARY KEY ((categoria), libro, usuario)
);''');

Agreamos informacion a la tabla

In [17]:
for row in df.iterrows():
    query = datos_a_insert_query('libros_por_categoria',row[1].to_dict(),
                                 columnas=['categoria', 'usuario','libro', 'calificacion'])            
    session.execute(query)

Realizamos la solución de la consulta:

In [18]:
def obtener_mejores_libros_categoria(categoria:str, top:int=5) -> list:
    """
    Regresa los mejores libros (mayor promedio de calificación) de una categoría dada

    Parametros
    ----------
    categoria : str
        La categoría en la que buscaremos los mejores libros

    top: int
        El número de mejores libros a regresar. Regresa 5 por defecto

    Regresa
    -------
    salida : list(str)
        Una lista de los mejores top libros por la categoría
    """
    query = f"""
    SELECT libro, AVG(calificacion) as promedio FROM libros_por_categoria
    WHERE categoria = '{categoria}'
    GROUP BY libro;
    """
    response = session.execute(query)
    mejores_libros = sorted(response.current_rows, key=lambda record: record.promedio, reverse=True)[:top]        
    return mejores_libros

Probamos nuestra funcion para obtener los mejores libros dada una categoria:

In [19]:
top = 4
mejores_libros = obtener_mejores_libros_categoria('cuento', top)

print(f"{' Mejores libros ':{'*'}{'^'}{30}}")
for i, libro in enumerate(mejores_libros):
    print(f'{i+1}. - {libro.libro} ({libro.promedio})')

******* Mejores libros *******
1. - the masque of the red death (10)
2. - the tell-tale heart (10)
3. - the fall of the house of usher (8)
4. - el tapiz amarillo (4)


**Resultado esperado**
~~~
******* Mejores libros *******
1. - the masque of the red death (10)
2. - the tell-tale heart (10)
3. - the fall of the house of usher (8)
4. - el tapiz amarillo (4)
~~~

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=dd92f94b-7239-46d3-b575-e515589eae55' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>